# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

image_name = '<YOUR_DOCKER_IMAGE>' # 'fboylu/kerastf-gpu' Feel free to use this image if you want to 
                                   # skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select 
                                                # the subscription you want to use here

In [1]:
resource_group = "fbakscpurg" # Feel free to modify these
aks_name = "fbAKSClustercpu"
location = "eastus"

image_name = "fboylu/kerasmnet-cpu" 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [2]:
!az login -o table

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FL7HK2GV9 to authenticate.
CloudName    IsDefault    Name                           State    TenantId
-----------  -----------  -----------------------------  -------  ------------------------------------
AzureCloud   False        Boston DS Dev                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Internal Consumption           Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   True         Team Danielle Internal         Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Core-ES-BLD                    Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Boston Engineering             Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        ADLTrainingMS                  Enabled  72f988bf-86f1-41af-91ab-2d7cd011db47
AzureCloud   False        Ads Eng Big Data Subscription  Enabled  72f988bf-86

In [7]:
!az account set --subscription $selected_subscription

In [4]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "fboylu@microsoft.com",
    "type": "user"
  }
}


You will also need to register the container service resources on your subscription if you haven't already done so.

In [9]:
!az provider register -n Microsoft.ContainerService

In [10]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [2]:
# !az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier.

In [3]:
%%time
# !az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_D4_v2

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10.5 µs


### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [57]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [5]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [9]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-1   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-2   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-3   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-4   Ready     agent     10d       v1.9.6


Let's check the pods on our cluster.

In [10]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          10d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          10d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          10d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          10d
kube-system   kube-proxy-225hz                        1/1       Running   0          10d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          10d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          10d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          10d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          10d
kube-system   kube-svc-redirect-278jk                 1/1       Running   0          10d
kube-system   kube-sv

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [11]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": "fboylu/kerasmnet-cpu",

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [12]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [13]:
write_json_to_file(app_template, 'az-dl.json')

In [14]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [15]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/kerasmnet-cpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
         

Next, we will use kubectl create command to deploy our application.

In [16]:
!kubectl create -f az-dl.json

deployment.apps "azure-dl" created
service "azure-dl" created


Let's check if the pod is deployed.

In [21]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-78c46df777-kpg5l               1/1       Running   0          3m
default       azure-dl-78c46df777-vwt47               1/1       Running   0          3m
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          10d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          10d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          10d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          10d
kube-system   kube-proxy-225hz                        1/1       Running   0          10d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          10d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          10d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          10d
kube-system   kube-prox

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [22]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                  SOURCE                              MESSAGE
5m          5m           1         azure-dl-56bf5dd85c-462vr.153993de5d047cc0   Pod          spec.containers{azure-dl}   Normal    Killing                 kubelet, aks-nodepool1-13347348-2   Killing container with id docker://azure-dl:Need to kill Pod
5m          5m           1         azure-dl-56bf5dd85c-46z7c.153993de5eab62ed   Pod          spec.containers{azure-dl}   Normal    Killing                 kubelet, aks-nodepool1-13347348-1   Killing container with id docker://azure-dl:Need to kill Pod
5m          5m           1         azure-dl-56bf5dd85c-5q4z8.153993de5e937219   Pod          spec.containers{azure-dl}   Normal    Killing                 kubelet, aks-nodepool1-13347348-0   Killing container with id docker://azure-dl:Need to kill Pod
5m          5m           1         azure-dl-5

Check the logs for the application pod.

In [23]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [24]:
!kubectl get pods

NAME                        READY     STATUS    RESTARTS   AGE
azure-dl-78c46df777-kpg5l   1/1       Running   0          3m
azure-dl-78c46df777-vwt47   1/1       Running   0          3m


In [25]:
{pod_dict['items'][0]['metadata']['name']}

{'azure-dl-78c46df777-kpg5l'}

In [26]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-06-19 13:52:01,555 CRIT Supervisor running as root (no user in config file)
2018-06-19 13:52:01,557 INFO supervisord started with pid 1
2018-06-19 13:52:02,560 INFO spawned: 'program_exit' with pid 9
2018-06-19 13:52:02,561 INFO spawned: 'nginx' with pid 10
2018-06-19 13:52:02,563 INFO spawned: 'gunicorn' with pid 11
2018-06-19 13:52:03,609 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-19 13:52:04.254661: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA

 2891776/17225924 [====>.........................] - ETA: 1s2018-06-19 13:52:07,588 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)

17227776/17225924 [==============================] - 1s 0us/step
{"logger": "model_driver", "path": "/code/driver.py", "tags": [], "timestamp": "2018-06-19T13:52:09.578938Z", "stack_info": 

In [51]:
!kubectl logs azure-dl-5bf5b784b8-nm746

2018-06-15 17:41:42,344 CRIT Supervisor running as root (no user in config file)
2018-06-15 17:41:42,347 INFO supervisord started with pid 1
2018-06-15 17:41:43,349 INFO spawned: 'program_exit' with pid 8
2018-06-15 17:41:43,350 INFO spawned: 'nginx' with pid 9
2018-06-15 17:41:43,352 INFO spawned: 'gunicorn' with pid 10
2018-06-15 17:41:44,384 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-06-15 17:41:44.787261: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-06-15 17:41:44.968350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1212] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: b5a8:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-06-15 17:41:44.968393: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1312] Adding visible gpu devices: 0
2018-06-15 1

In [45]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   35        35        35           35          46m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [35]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.215.15   13.68.134.150   80:32673/TCP   6m


# Scaling

In [29]:
# !az aks scale --resource-group=$resource_group --name=$aks_name --node-count 2

In [30]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-1   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-2   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-3   Ready     agent     10d       v1.9.6
aks-nodepool1-13347348-4   Ready     agent     10d       v1.9.6


In [36]:
!kubectl scale --current-replicas=2 --replicas=20 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [46]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-78c46df777-4gvvk               1/1       Running   0          36m
default       azure-dl-78c46df777-5ln72               1/1       Running   0          40m
default       azure-dl-78c46df777-6r5s4               1/1       Running   0          40m
default       azure-dl-78c46df777-6vcvb               1/1       Running   0          36m
default       azure-dl-78c46df777-7ng74               1/1       Running   0          40m
default       azure-dl-78c46df777-884k7               1/1       Running   0          40m
default       azure-dl-78c46df777-88t4j               1/1       Running   0          40m
default       azure-dl-78c46df777-chgnb               1/1       Running   0          36m
default       azure-dl-78c46df777-f4ph6               1/1       Running   0          40m
default       azure-dl-78c46df777-fg247               1/1       Running   0          40m
default       azure-d

In [42]:
!kubectl scale --current-replicas=20 --replicas=35 deployment/azure-dl

deployment.extensions "azure-dl" scaled


In [37]:
# !az aks scale --resource-group=$resource_group --name=$aks_name --node-count 1

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [165]:
!kubectl delete -f az-dl.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [169]:
!az aks delete -n $aks_name -g $resource_group -y

inished ..

In [170]:
!az group delete --name $resource_group -y

inished ..